In [1]:
# Let's start with the basics:
2 + 2

4

In [2]:
# But what does "2 + 2" actually mean in Julia?
# Let's save that expression. The syntax :() lets
# us capture a paresed expression as a Julia object:
ex = :(2 + 2)

:(2 + 2)

In [3]:
# We can look at the expression in more detail with dump()
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 2
    3: Int64 2
  typ: Any


In [4]:
# So :(2 + 2) is a `call` expression with the arguments `+`, `2`, and `2`. 
# We can construct that expression by hand if we want:
ex2 = Expr(:call, :+, 2, 2)

:(2 + 2)

In [6]:
# That's all there is to it. Those two expressions are the same:
ex2 == ex

true

In [9]:
# We can make expressions that are arbitrarily complicated by using
# the Expr() constructor. Let's generate a nested function call:
Expr(:call, :f, Expr(:call, :+, 2, 2))

:(f(2 + 2))

In [10]:
# To evaluate an expression, we use eval():
ex = :(2 + 2)
eval(ex)

4

In [11]:
# An Expr in Julia is always constructed from parsed data. So it's
# impossible to have an Expr that corresponds to a syntax error. 
# That means that when you're metaprogramming in Julia, you'll never
# accidentally produce code that can't be parsed. 

# For example, we can't construct an expression with an incomplete statement,
# so the code below will fail:
:(2 + )

LoadError: [91msyntax: unexpected )[39m

In [41]:
# We can write functions to transform Julia expressions into other
# expressions. 

# # For example, let's create a really useless function that changes
# every `x` variable in an expression into a `y`:
# (note: we put a ! at the end of the function name because
# it mutates its input argument. That's not a rule in Julia,
# but it's a pretty widely followed convention).
function x_to_y!(expr::Expr)
    # For each argument in the expression:
    for i in 1:length(expr.args)
        # If that argument is the symbol :x
        if expr.args[i] == :x
            # Then replace it with the symbol :y
            expr.args[i] = :y
        # Otherwise, if that argument is another expression...
        elseif isa(expr.args[i], Expr)
            # Then recurse into that expression and change all its x's too
            x_to_y!(expr.args[i])
        end
    end
    return expr
end


x_to_y! (generic function with 1 method)

In [32]:
# Let's see it in action:
x_to_y!(:(2 + x))

:(2 + y)

In [42]:
# And let's see it act recursively:
x_to_y!(:(f(x * (2 + x))))

:(f(y * (2 + y)))

In [43]:
# Since the expression is already parsed by the time it's passed to
# `x_to_y!()`, we don't have to worry about running any crazy regexes
# to figure out exactly what corresponds to the variable `x`. For example,
# our function will never be confused by some other variable that just 
# happens to contain an x:
x_to_y!(:(x1 + 1))

:(x1 + 1)

In [35]:
# A macro in Julia is just a function that takes in parsed Julia
# expressions and returns a new expression to be substituted into
# the program. Whatever the macro returns will then be executed
# when the program is run. 

# For example, let's write a macro that just calls our x_to_y function:

macro x_to_y(expr)
    return x_to_y!(expr)
end


@x_to_y (macro with 1 method)

In [36]:
# Now let's use our macro! First we'll define the variable y:
y = 2

2

In [37]:
# Trying to do some computation with x will fail, since x isn't yet defined:
x + 1

LoadError: [91mUndefVarError: x not defined[39m

In [38]:
# But we can use our macro to turn that x into a y by magic!
@x_to_y x + 1

3

In [39]:
# In case it's not clear what the macro did, we can use
# macroexpand() to show it:
macroexpand(:(@x_to_y x + 1))

:(Main.y + 1)

In [40]:
# And since our x_to_y! function is recursive, the macro will
# work on arbitrarily nested expressions:
@x_to_y x + 3 * (x + 2 / (x - 1))

14.0

It's just that easy! For more on metaprogramming in Julia, 
check out the manual: <https://docs.julialang.org/en/stable/manual/metaprogramming/>
